In [13]:
parsed_file = "test_file"
word_occurence_map = {}

with open(parsed_file) as inp:
    prev_line, prev_count = '', 0
    for line in inp:
        curr_line = "\t".join(line.strip().split("\t")[:-1])
        if curr_line == prev_line or (not prev_line):
            prev_count += int(line.strip().split("\t")[-1])
        else:
            print (curr_line)
            x, y, path = prev_line.split('\t')
            key = str(x) + '_' + str(y)
            current = path + ":" + str(prev_count)
            if key in word_occurence_map:
                pastkeys = word_occurence_map[key]
                current =  pastkeys + ',' + current
            word_occurence_map[key] = current
            prev_count = 1
        prev_line = curr_line
    x, y, path = prev_line.split('\t')
    key = str(x) + '_' + str(y)
    current = path + ":" + str(prev_count)
    if key in word_occurence_map:
        pastkeys = word_occurence_map[key]
        current =  pastkeys + ',' + current
    word_occurence_map[key] = current
    


19193456	23452389	9668810
663726	9048846	4107215
6178566	6097931	8733868
8184873	13389052	6536081
5933740	15640897	8593453
1837912	2946514	9443662
16864707	22008180	11071817
13583494	15294065	657327
18679314	8042043	11954734
18679314	8042040	11954736
18679314	8042043	11954739


In [23]:
import sys, shelve, picklw
file = "damru"
p = pickle.load(open(file, 'rb'))
db_file = ".".join(file.split(".")[:-1]) + ".db" 
with shelve.open(db_file, 'c') as db:
    for k in p:
        try:
            p[k] = p[k].decode("utf-8")
        except:
            pass
        try:
            k = k.decode("utf-8")
        except:
            pass

        if k and p[k]:
            db[str(k)] = str(p[k])
print ("db_file", db_file, "done")

KeyboardInterrupt: 

In [ ]:
import os,shelve
folders = ["../junk/Files/" + l + "/" for l in os.listdir("../junk/Files/") if l.startswith("security_threshold_")]

passed1 = []
for f in folders:
    if len(list(shelve.open(f + "security_relations_map.db", "r").items())):
        passed1.append(f)

print ("Folders having uncorrupted relation dict {}/{}".format(len(passed1), len(folders)))

passed2 = []
for f in passed1:
    if len(list(shelve.open(f + "security_path_to_id_dict.db", "r").items())) and len(list(shelve.open(f + "security_id_to_path_dict.db", "r").items())):
        passed2.append(f)

print ("Folders having uncorrupted path-to-id dict {}/{}".format(len(passed2), len(folders)))

passed3,passed4, passed5 = [], [], []

for f in passed2:
    s1 = list(shelve.open(f + "security_id_to_word_dict.db", "r").keys())
    s2 = list(shelve.open(f + "security_word_to_id_dict.db", "r").values())
    if len(s1) and len(s2):
        passed3.append(f)
        s1_sort = sorted(s1, key=lambda l:float(l))
        s2_sort = sorted(s2, key=lambda l:float(l))
        if s1_sort[0] == 0 and s2_sort[0] == 0:
            passed4.append(f)
        if s1_sort == s2_sort:
            passed5.append(f)


print ("Folders having uncorrupted word-to-id dict {}/{}".format(len(passed3), len(folders)))
print ("Folders having zero-indexed word-to-id dict {}/{}".format(len(passed4), len(folders)))
print ("Folders having equal word-to-id and id-to-word dicts {}/{}".format(len(passed5), len(folders)))



In [7]:
print ("Folders having uncorrupted relation dict {}/{}".format(2, 4))

Folders having uncorrupted relation dict 2/4


In [11]:
dataset = open("../files/dataset/dataset.tsv").read().split("\n")
dataset_t = [l for l in dataset if l.split("\t")[-1]!="none"]
dataset_f = [l for l in dataset if l.split("\t")[-1]=="none"]
import random
dataset_t_mini = random.choices(dataset_t, k=int(0.1 * len(dataset_t)))
dataset_f_mini = random.choices(dataset_f, k=10000-int(0.1 * len(dataset_t)))
dataset_mini = dataset_t_mini + dataset_f_mini
random.shuffle(dataset_mini)
open("../files/dataset/dataset_mini.tsv","w+").write("\n".join(dataset_mini))
open("../files/dataset/train_mini.tsv","w+").write("\n".join(dataset_mini[:int(0.9*len(dataset_mini))]))
open("../files/dataset/test_mini.tsv","w+").write("\n".join(dataset_mini[int(0.9*len(dataset_mini)):]))

33268

In [26]:
from pronto import Ontology
import re

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0).lower() for m in matches])

ont = Ontology("/Users/vivek/SIREN-Research/securityontology.owl")
ontology = [camel_case_split(a.id.split("#")[-1])+"\tcontrol\thypernym" for a in ont.terms()]
import itertools

ontology = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in ontology))

In [46]:
lines = [l for l in open("temp_policy","r").read().split("\n") if "policy" in l.split("\t")[0]]
final = []
for l in lines:
    final.append(l.split("\t")[0] + "\tpolicy\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tpolicy\thypernym")
    final.append(l.split("\t")[0] + "\tintangible asset\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tintangible asset\thypernym")
    final.append(l.split("\t")[0] + "\tasset\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tasset\thypernym")
    

In [47]:
open("../junk/policy","w+").write("\n".join(final))

15077

In [52]:
lines = [l.split("\t") for l in open("../files/dataset/test_knocked.tsv","r").read().split("\n")]
final_lines = []
for l in lines:
    final_lines.append("\t".join(l))
    final_lines.append("\t".join(l[:2][::-1] + [l[2].replace("concept", "instance").replace("hypernym", "hyponym")]))
final_lines = sorted(list(set(final_lines)))
open("../files/dataset/test_knocked.tsv","w+").write("\n".join(final_lines))

287258

In [51]:
len(final_lines), len(list(set(final_lines)))

(5910, 5562)